## Projet de Machine Learning 

In [1014]:
import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
import joblib
spark=SparkSession.builder.master("local").appName("TP_MLFE").getOrCreate()
import sys
from pyspark.sql.functions import *

In [973]:
Dataset1 = spark.read.format("csv").option("sep",",").option("inferSchema", "true").option("header", "true").load("ccdefault.csv").withColumnRenamed("PAY_0", "PAY_1").withColumnRenamed("DEFAULT", "label")

In [974]:
from pyspark.sql.functions import isnan, when, count, col

In [975]:
Dataset1.select([count(when(isnan(c), c)).alias(c) for c in rawCreditCardDataset.columns]).show()

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|  0|        0|  0|        0|       0|  0|    0|    0|    0|    0|    0|    0|        0|        0|        0|        0|        0|        0|       0|       0|       0|       0|       0|       0|    0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+



In [976]:
Dataset2 = Dataset1.withColumn("EDUCATION", when(col("EDUCATION") > 3, 0).otherwise(col("EDUCATION"))).withColumn("MARRIAGE", when(col("MARRIAGE") == 0, 3).otherwise(col("MARRIAGE")))

In [977]:
for i in range(1,7):
    colName = "PAY_"+str(i)
    Dataset2 = Dataset2.withColumn(colName, when(col(colName) < 0, 0).otherwise(col(colName)))

In [978]:
Dataset2.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------+-----------------+------------------+-----------------+----------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+-------------------+
|summary|               ID|         LIMIT_BAL|               SEX|         EDUCATION|          MARRIAGE|              AGE|             PAY_1|              PAY_2|              PAY_3|              PAY_4|             PAY_5|              PAY_6|        BILL_AMT1|        BILL_AMT2|        BILL_AMT3|         BILL_AMT4|        BILL_AMT5|       BILL_AMT6|         PAY_AMT1|          PAY_AMT2|         PAY_AMT3|          PAY_AMT4|          PAY_AMT5|         PAY_AMT6|              label|
+-------+-----------------+---------------

In [979]:
Dataset2.groupBy("EDUCATION").agg(avg("LIMIT_BAL").alias("LIMIT_BAL_MEAN")).show()

+---------+------------------+
|EDUCATION|    LIMIT_BAL_MEAN|
+---------+------------------+
|        1|212956.06991025034|
|        3|126550.27049013626|
|        2| 147062.4376336422|
|        0| 181316.2393162393|
+---------+------------------+



In [980]:
Dataset2.groupBy("label", "SEX").count().sort(asc("label"), asc("SEX")).show()

+-----+---+-----+
|label|SEX|count|
+-----+---+-----+
|    0|  1| 9015|
|    0|  2|14349|
|    1|  1| 2873|
|    1|  2| 3763|
+-----+---+-----+



In [981]:
Dataset2.describe("LIMIT_BAL", "AGE", "PAY_1").show()

+-------+------------------+-----------------+------------------+
|summary|         LIMIT_BAL|              AGE|             PAY_1|
+-------+------------------+-----------------+------------------+
|  count|             30000|            30000|             30000|
|   mean|167484.32266666667|          35.4855|0.3567666666666667|
| stddev|129747.66156720246|9.217904068090155|0.7605941727834076|
|    min|             10000|               21|                 0|
|    max|           1000000|               79|                 8|
+-------+------------------+-----------------+------------------+



In [982]:
Dataset2.groupBy("label", "MARRIAGE").count().sort(asc("label"), asc("MARRIAGE")).show()

+-----+--------+-----+
|label|MARRIAGE|count|
+-----+--------+-----+
|    0|       1|10453|
|    0|       2|12623|
|    0|       3|  288|
|    1|       1| 3206|
|    1|       2| 3341|
|    1|       3|   89|
+-----+--------+-----+



In [983]:
Dataset2.groupBy("label", "EDUCATION").count().sort(asc("label"), asc("EDUCATION")).show()

+-----+---------+-----+
|label|EDUCATION|count|
+-----+---------+-----+
|    0|        0|  435|
|    0|        1| 8549|
|    0|        2|10700|
|    0|        3| 3680|
|    1|        0|   33|
|    1|        1| 2036|
|    1|        2| 3330|
|    1|        3| 1237|
+-----+---------+-----+



In [984]:
from pyspark.ml.feature import VectorAssembler,StandardScaler
from pyspark.ml.feature import OneHotEncoder
va = VectorAssembler().setInputCols(["LIMIT_BAL", "AGE", "EDUCATION", "SEX", "MARRIAGE", "label"]).setOutputCol("feature")                                 

In [985]:
numericColonn = ["LIMIT_BAL"]

In [986]:
categoricColonns = ["EDUCATION", "SEX", "MARRIAGE", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]

In [987]:
categoricColumnsOhe = list(map(lambda x :x+"_OHE",categoricColumns))

In [988]:
va = VectorAssembler().setInputCols(numericColonn).setOutputCol("numerical_attributes")

In [989]:
scaler = StandardScaler().setInputCol("numerical_attributes").setOutputCol("scaled_attributes")

In [990]:
encoder = OneHotEncoder().setInputCols(categoricColonns).setOutputCols(categoricColonnsOhe)

## Creation des Pipeline

In [991]:
from pyspark.ml import Pipeline, PipelineModel

In [992]:
Pipeline1 =  Pipeline().setStages([va, scaler])
Pipeline2 =  Pipeline().setStages([encoder])
pipeline =  Pipeline().setStages([Pipeline1, Pipeline2])
DatasetF = pipeline.fit(creditCardDataset).transform(creditCardDataset)

## Creation du dataset Final

In [993]:
datasetFINAL = VectorAssembler().setInputCols([("scaled_attributes")] + categoricColonnsOhe).setOutputCol("features").transform(DatasetF).select("features", "label")

In [994]:
datasetFINAL.show(5, False)

+-----------------------------------------------------------------------------------------+-----+
|features                                                                                 |label|
+-----------------------------------------------------------------------------------------+-----+
|(57,[0,3,7,11,19,25,33,41,49],[0.15414535998894324,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])     |1    |
|(57,[0,3,8,9,19,25,33,41,51],[0.9248721599336596,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |1    |
|(57,[0,3,8,9,17,25,33,41,49],[0.6936541199502446,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |0    |
|(57,[0,3,7,9,17,25,33,41,49],[0.38536339997235813,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])      |0    |
|(57,[0,3,5,7,9,17,25,33,41,49],[0.38536339997235813,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0    |
+-----------------------------------------------------------------------------------------+-----+
only showing top 5 rows



In [995]:
trainDS, testDS = datasetFINAL.randomSplit([0.8, 0.2])

In [996]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.sql import DataFrame
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Regression Logistique

In [931]:
lr = LogisticRegression().setMaxIter(10)

In [932]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam,[0.2, 0.1, 0.0]).addGrid(lr.elasticNetParam,[0.2, 0.1, 0.0]).build()

In [933]:
evaluator = BinaryClassificationEvaluator()

In [934]:
v = CrossValidator().setEstimator(lr).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(4)

In [935]:
lrModel = v.fit(trainDS)

In [936]:
best_model_lr = lrModel.bestModel.summary

In [937]:
predictions = lrModel.transform(testDS)

In [938]:
predictions.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  927|
|    0|       0.0| 4511|
|    1|       1.0|  386|
|    0|       1.0|  185|
+-----+----------+-----+



In [939]:
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.7662158554670837


In [944]:
acc=(4511+386)/(927+4511+386+185)
acc

0.8149442502912299

In [941]:
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')

In [942]:
my_mc_lr.evaluate(predictions)

0.7852775910661873

## Arbre de decision

In [997]:
from pyspark.ml.classification import DecisionTreeClassifier

In [998]:
dt = DecisionTreeClassifier()

In [999]:
paramGrid =ParamGridBuilder().addGrid(dt.maxDepth, [4, 5, 6, 7, 8, 9, 10]).build()

In [1000]:
evaluator = BinaryClassificationEvaluator()

In [1001]:
cv = CrossValidator().setEstimator(dt).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(4)

In [1002]:
dtModel = cv.fit(trainDS)

In [1003]:
predictions = dtModel.transform(testDS)

In [1008]:
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.35656290126833085


In [1005]:
predictions.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  898|
|    0|       0.0| 4430|
|    1|       1.0|  463|
|    0|       1.0|  265|
+-----+----------+-----+



In [1007]:
(4430+463)/(4430+463+898+265)

0.8079590488771466

In [955]:
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')

In [956]:
my_mc_lr.evaluate(predictions)

0.7946836065641139

## RandomForest

In [957]:
from pyspark.ml.classification import RandomForestClassifier

In [958]:
rf = RandomForestClassifier()

In [959]:
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth,[10, 15]).addGrid(rf.featureSubsetStrategy,["all", "sqrt"]).addGrid(rf.numTrees,[10, 15,20]).build()

In [960]:
valuator = BinaryClassificationEvaluator()

In [961]:
cv = CrossValidator().setEstimator(rf).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(4)

In [962]:
rfModel = cv.fit(trainDS)

In [963]:
predictions = rfModel.transform(testDS)

In [964]:
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.7710553357786315


In [965]:
predictions.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  837|
|    0|       0.0| 4449|
|    1|       1.0|  476|
|    0|       1.0|  247|
+-----+----------+-----+



In [969]:
(4449+476)/(837+4449+476+247)

0.81960392744217

In [970]:
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')

In [971]:
my_mc_lr.evaluate(predictions)

0.7987972700659345